In [1]:
import import_ipynb
import sys
import torch
from torch import nn
from torch import optim

if torch.cuda.is_available():
    cuda = torch.cuda.device("cuda:0")

%cd /home/ankit/Commitments/earthquake-prediction/

from preprocessing import *
from training import *
from postprocessing import *
from testing import *

%cd /home/ankit/Commitments/earthquake-prediction/Indonesia-Sulawesi/

/home/ankit/Commitments/earthquake-prediction
importing Jupyter notebook from preprocessing.ipynb
importing Jupyter notebook from training.ipynb
torch.Size([10, 1, 9])
torch.Size([1, 9])
torch.Size([10, 1, 9])
torch.Size([1, 9])
torch.Size([10, 1, 9])
torch.Size([1, 90])
importing Jupyter notebook from postprocessing.ipynb
importing Jupyter notebook from testing.ipynb
/home/ankit/Commitments/earthquake-prediction/Indonesia-Sulawesi


In [2]:
# SETTINGS D

data_file = "data.csv"
time_parser = time_from_string_dmy
mag_range = (5, 6)

rows = 4
cols = 4
total_params = rows*cols
time_intervals = 690
train_size = 0.8
lookback_window = 50

In [3]:
earthquakes = extract_from_csv(data_file, time_parser, mag_range)
cells = Spatio_Temporal_Segmentation(rows, cols, time_intervals, earthquakes)

print(cells[0][0][0])

No earthquakes were recorded in this cell.


In [4]:
generator = TensorGen(cells)

eventtensor = generator.eventNoEvent()
freqtensor = generator.freq()
minmagtensor = generator.min_magnitude()
maxmagtensor = generator.max_magnitude()

if torch.cuda.is_available:
    eventtensor = eventtensor.cuda()
    freqtensor = freqtensor.cuda()
    minmagtensor = minmagtensor.cuda()
    maxmagtensor = maxmagtensor.cuda()
    
print(eventtensor.size())
print(freqtensor.size())
print(minmagtensor.size())
print(maxmagtensor.size())

torch.Size([690, 1, 16])
torch.Size([690, 1, 16])
torch.Size([690, 1, 16])
torch.Size([690, 1, 16])


<strong>TRAINING EVENT PREDICTOR</strong>

In [18]:
in_channels = lookback_window
out_channels = 50
kernel_size = 3
in_parameters = rows*cols 

In [19]:
model = CNN(in_channels, out_channels, kernel_size, in_parameters, activation=nn.ReLU())

if torch.cuda.is_available():
    model = model.cuda()
    
# out = model(eventtensor[:100])
# print(out)                    

In [20]:
# SETTINGS T1
pos_weight = torch.tensor([2])
pos_weight = pos_weight.cuda()
                          
loss_function = nn.BCEWithLogitsLoss(pos_weight=pos_weight)
n_epochs = 100

In [24]:
optimizer = optim.Adagrad(model.parameters(), lr=0.001)

# tqdm._instances.clear()
train(model, eventtensor, optimizer, loss_function, n_epochs=1, lookback_window=lookback_window)

100%|██████████| 640/640 [00:03<00:00, 206.21it/s]

Epoch 1 completed. 
Training Loss: 0.811
Test Loss: 0.844


In [25]:
evaluate(data=eventtensor, model=model, train_size=0.8, lookback_window=lookback_window, threshold=0.5)

100%|██████████| 640/640 [00:01<00:00, 496.59it/s]


Evaluation results for training dataset:
8032 evaluations in total.
723/2362 cells with earthquake activity were correctly identified. (30.61 %)
5601/5670 cells with no earthquake activity were correctly identified. (98.78 %)
69/792 positives turned out to be false.
1639/7240 negatives turned out to be false.
Preicision: 0.913
Recall: 0.306
Overall accuracy: 78.74 %
Evaluation results for training dataset:
2208 evaluations in total.
155/577 cells with earthquake activity were correctly identified. (26.86 %)
1565/1631 cells with no earthquake activity were correctly identified. (95.95 %)
66/221 positives turned out to be false.
422/1987 negatives turned out to be false.
Preicision: 0.701
Recall: 0.269
Overall accuracy: 77.90 %
